In [3]:
import keras
import os 
from mtcnn import MTCNN
from matplotlib import pyplot
import numpy as np
from matplotlib.patches import Rectangle
import cv2
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import *
from keras import Sequential
from keras.regularizers import *
from keras.applications import vgg19

In [4]:
TRAIN_DIRECTORY = "D:\\6th Sem\\AI\\ai"
detector = MTCNN()

In [5]:
def detect_faces(img):
    face = detector.detect_faces(img)
    if len(face) == 0:
        return face
    x, y, width, height = face[0]['box']
    return img[y:y+height+50 , x:x+width+50, :]

In [6]:
users = {'Aman' : 0, 'Arjun' : 1, 'Divij' : 2, 'Soumya' : 3}
images = []
label = []

In [8]:
for file in os.listdir(TRAIN_DIRECTORY):
    path = os.path.join(TRAIN_DIRECTORY,file)
    for img1 in os.listdir(path):
        img_path = os.path.join(path, img1)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = detect_faces(img)
        if(img.shape[0] == 0 or img.shape[1] == 0 or len(img) == 0):
            continue
        img = cv2.resize(img,(224,224))/255
        images.append(img)
        label.append(users[file])

In [9]:
df = pd.DataFrame()
df['image'] = images
df['label'] = label
X_train=np.array(df.iloc[:,0].tolist())
y_train = np.array(df.iloc[:,1].tolist())
y_train = to_categorical(y_train)

In [17]:
source_model = vgg19.VGG19(weights = 'imagenet')
drop_layer = Dropout(0.5)
drop_layer2 = Dropout(0.5)
model = Sequential()
for layer in source_model.layers[:-1]:
    if layer == source_model.layers[-25]:
        model.add(BatchNormalization())
    model.add(layer)
model.add(Dense(4, activation="softmax"))
model.summary()

574726144/574710816 [==============================] - 181s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________

In [18]:
from keras.optimizers import RMSprop
lr = 1e-5
decay = 1e-7 #0.0
optimizer = RMSprop(lr=lr, decay=decay)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [19]:
history = model.fit(X_train,y_train,steps_per_epoch = 20,shuffle = True,epochs = 6)

Epoch 1/6
20/20 [==============================] - 59s 3s/step - loss: 3.5411 - accuracy: 0.2822
Epoch 2/6
20/20 [==============================] - 48s 2s/step - loss: 0.5434 - accuracy: 0.7082
Epoch 3/6
20/20 [==============================] - 48s 2s/step - loss: 0.0371 - accuracy: 1.0000
Epoch 4/6
20/20 [==============================] - 16s 745ms/step - loss: 0.0130 - accuracy: 1.0000


In [21]:
user2 = {0:"Aman",1:"Arjun",2:"Divij",3:"Soumya"}
temp = []
for file in os.listdir("D:\\6th Sem\\AI\\test"):
    img_path = os.path.join(r'D:\6th Sem\AI\test', file)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = detect_faces(img)
    if(img.shape[0] == 0 or img.shape[1] == 0 or len(img) == 0):
        continue
    img = cv2.resize(img,(224,224))/255.0
    temp.append(img)
    val = model.predict(np.array(temp))
    cla = np.argmax(val, axis=1)
    print(user2[cla[0]])
    print(val)
    temp = []

Soumya
[[0.04760016 0.12840787 0.04583082 0.7781611 ]]
